In [59]:
# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-h1rjQ-5s3gzmiRKtg7ZWm7UmNuw5adp4qYoCGugeCg&tc=bKXz5EtWEcBOOHFkcspBdmfeN1da0NVblUYc7HLyIFQ&cc=B3_tcj4D4y2RlJ0WnMWnxts_zDmKpry1Hcicr0a5Rw8

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjktsBFHVf0totrn9WXEABCGs4p1P0461lPLtmEJBjvAxhipEHFncA

Successfully saved authorization token.


In [4]:
!pip install earthengine-api
!pip install geopandas
import os
import sys
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 16.7 MB 1.3 MB/s 
     |████████████████████████████████| 6.3 MB 38.6 MB/s 


In [178]:
import ee
import pandas as pd
import numpy as np
import geopandas as gpd

In [ ]:
## start here for running from csv

In [185]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [186]:
path = '/content/drive/MyDrive/tailings/az_positives.csv'
df_pos = pd.read_csv(path)

In [187]:
#run this cell for positives csv for cleaning
df_pos['clean'] = df_pos.tailingsCentroid.str[31:-2]
df_pos['class'] = 'positive'
df_pos[['long', 'lat']] = df_pos['clean'].str.split(',', 2, expand=True)
df_pos = df_pos[['lat', 'long', 'class']]
df_pos['lat'] = df_pos['lat'].astype('float64')
df_pos['long'] = df_pos['long'].astype('float64')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [249]:
#code to extract coordinates
def offset(lat,lon,x,y):

	#Earth’s radius, sphere
	R=6378137

	#Coordinate offsets in radians
	dLat = x/R
	dLon = y/(R*math.cos(math.pi*lat/180))

	return lat + dLat * 180/math.pi, lon + dLon * 180/math.pi
 

def exportImage(data,size=0):
  # Loop the csv file.
  for d in range(data.shape[0]):
    x = data[d][0]
    y = data[d][1]
    
    llx , lly = offset(x,y,-size/2,-size/2)
    urx , ury = offset(x,y,size/2,	size/2)

    label = data[d][2]

    geom_raw = [[lly,llx], [ury,llx], [ury,urx], [lly,urx]]

    geom = ee.Geometry.Polygon(geom_raw)


    collection = (ee.ImageCollection("COPERNICUS/S2")
              # Select the Red, Green and Blue image bands, as well as the cloud masking layer.
              .select(['B4', 'B3', 'B2', 'QA60'])
              # Filter for images within a given date range.
              .filter(ee.Filter.date('2022-01-01', '2022-06-01'))
              # Filter for images that overlap with the assigned geometry.
              .filterBounds(geom)
              # Filter for images that have less then 20% cloud coverage.
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
              # Apply cloud mask.
              .map(maskS2clouds)
              )
    # Sort images in the collection by index (which is equivalent to sorting by date), 
    # with the oldest images at the front of the collection.
    # Convert collection into a single image mosaic where only images at the top of the collection are visible.
    image = collection.sort('system:index', opt_ascending=False).mosaic()

    # Assign visualization parameters to the image.
    image = image.visualize(bands=['B4', 'B3', 'B2'],
                          min=[0.0, 0.0, 0.0],
                          max=[0.3, 0.3, 0.3]
                        )



    # Assign export parameters.
    task_config = {
      'region': geom,
      'folder': 'neg_2km',
      'scale': 10,
      'crs': 'EPSG:4326',
      'description': label + str(d)
      }

    # Export Image
    task = ee.batch.Export.image.toDrive(image, **task_config)
    task.start()

In [250]:
data = df_city.values
exportImage(data,2000)

Creating negatives file:

In [191]:
from  geopy.geocoders import Nominatim

In [239]:
path = '/content/drive/MyDrive/tailings/az_cities.csv'
df_city = pd.read_csv(path)

In [240]:
df_city

,city
0,Phoenix
1,Tucson
2,Mesa
3,Chandler
4,Scottsdale
...,...
242,Arivaca
243,San Jose
244,Ash Fork
245,East Fork


In [195]:
# city_list = df_city.city.values.tolist()

In [197]:
# city_list[:5]

['Phoenix', 'Tucson', 'Mesa', 'Chandler', 'Scottsdale']

In [223]:
df_city['city'][:5]

0       Phoenix
1        Tucson
2          Mesa
3      Chandler
4    Scottsdale
Name: city, dtype: object

In [235]:
#test
geolocator = Nominatim()
city = 'Teec Nos Pos'
state = "Arizona"
loc = geolocator.geocode(city+','+ state)
print("latitude is :" ,loc.latitude,"\nlongtitude is:" ,loc.longitude)

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


latitude is : 36.918902 
longtitude is: -109.07962728700835


In [200]:
# df_neg = pd.DataFrame()
# df_neg['place'] = city_list
# # df_neg['type'] = 'city'

In [207]:
# df_neg

,place
0,Phoenix
1,Tucson
2,Mesa
3,Chandler
4,Scottsdale
...,...
245,Arivaca
246,San Jose
247,Ash Fork
248,East Fork


In [ ]:
# #suburbs
# df2 = pd.DataFrame()
# df2['place'] = ['Safety Harbor', 'Vero Beach', 'Venice', 'Apalachicola', 'Captiva', 'Crystal River', 'Pompano Beach', 'Jupiter', 'Dunedin', 'Sebring', 'Winter Park', 'Anna Maria Island',
#                 'Fernandina Beach', 'Islamorada', 'Coconut Creek', 'Coral Springs', 'Mount Dora', 'Matlacha', 'Havana', 'Bristol']
# df2['type'] = 'suburbs'

In [ ]:
# df3 = pd.DataFrame()
# df3['place'] = ['Lake City', 'Lady Lake', 'Lake Mary', 'Lauderdale Lakes', 'Lake Worth', 'Lake Alfred', 'Lake Park', 'Lakeland', 'Lake Wales', 'Groveland']
# df3['type'] = 'lakes'

In [ ]:
# df4 = pd.DataFrame()
# df4['place'] = ['La Crosse', 'Sopchoppy', 'McIntosh', 'Hillcrest Heights', 'Ebro', 'Cinco Bayou', 'Highland Park', 'Brooker', 'Manalapan', 'Otter Creek']
# df4['type'] = 'least_pop'

In [ ]:
# fulldf = pd.concat([df, df2, df3, df4], ignore_index=True)
# fulldf

In [241]:
lat = []
long = []
for item in df_city['city']:
  geolocator = Nominatim()
  city = item
  state = "Arizona"
  loc = geolocator.geocode(city + ',' + state)
  lat.append(loc.latitude)
  long.append(loc.longitude)

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [244]:
df_city['lat'] = lat
df_city['long'] = long
df_city['class'] = 'negative'
df_city = df_city[['lat', 'long', 'class']]
df_city['lat'] = df_city['lat'].astype('float64')
df_city['long'] = df_city['long'].astype('float64')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# fulldf.to_csv('/content/drive/My Drive/fl_training.csv', encoding='utf-8', index=False)

Scratchwork:

In [60]:
# use this for testing one-off coordinates


# This is the cloud masking function provided by GEE but adapted for use in Python.
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    mask = mask.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)


# Define the geometry of the area for which you would like images

# geom1 =[[33.8777, -13.4055], [33.8777, -13.3157], [33.9701, -13.3157], [33.9701, -13.4055]]

#test az coordinates that works
# geom1 = [[-112.0068, 33.4268], [-111.8452, 33.4268], [-111.8452, 33.5615], [-112.0068, 33.5615]]

# geom1 =[[-112.00680781469596, 33.426845253691035], [-111.84522898530405, 33.426845253691035], [-111.84522898530405, 33.561592546308965], [-112.00680781469596, 33.561592546308965]]

#test positive polygons

geom1 =[[-110.74634974491924,33.077723055754134] , [-110.74460172576156,33.077723055754134], [-110.74460172576156,33.079230279423406], [-110.74634974491924,33.079230279423406]]

geom = ee.Geometry.Polygon(geom1)

# Call collection of satellite images.
collection = (ee.ImageCollection("COPERNICUS/S2")
              # Select the Red, Green and Blue image bands, as well as the cloud masking layer.
              .select(['B4', 'B3', 'B2', 'QA60'])
              # Filter for images within a given date range.
              .filter(ee.Filter.date('2017-01-01', '2017-03-31'))
              # Filter for images that overlap with the assigned geometry.
              .filterBounds(geom)
              # Filter for images that have less then 20% cloud coverage.
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
              # Apply cloud mask.
              .map(maskS2clouds)
             )

# Sort images in the collection by index (which is equivalent to sorting by date), 
# with the oldest images at the front of the collection.
# Convert collection into a single image mosaic where only images at the top of the collection are visible.
image = collection.sort('system:index', opt_ascending=False).mosaic()


# Assign visualization parameters to the image.
image = image.visualize(bands=['B4', 'B3', 'B2'],
                        min=[0.0, 0.0, 0.0],
                        max=[0.3, 0.3, 0.3]
                       )

# Assign export parameters.
task_config = {
    'region': geom.coordinates().getInfo(),
    'folder': 'tailings',
    'scale': 10,
    'crs': 'EPSG:4326',
    'description': 'test_format',
}

# Export Image
task = ee.batch.Export.image.toDrive(image, **task_config)
task.start()

In [ ]:
##pull negatives for arizona, nevada, new mexico

In [ ]:
def offset(lat,lon,x,y):

	#Earth’s radius, sphere
	R=6378137

	#Coordinate offsets in radians
	dLat = x/R
	dLon = y/(R*math.cos(math.pi*lat/180))

	return lat + dLat * 180/math.pi, lon + dLon * 180/math.pi

In [ ]:
def exportImage(data,size=0):
  # Loop the csv file.
  for d in range(data.shape[0]):
    x = data[d][0]
    y = data[d][1]
    
    llx , lly = offset(x,y,-size/2,-size/2)
    urx , ury = offset(x,y,size/2,	size/2)

    label = data[d][2]

    geom_raw = [[lly,llx], [ury,llx], [ury,urx], [lly,urx]]

    geom = ee.Geometry.Polygon(geom_raw)
    
    print(geom_raw)

In [ ]:
data = df_bonus.values
exportImage(data,15000)

[[-112.00680781469596, 33.426845253691035], [-111.84522898530405, 33.426845253691035], [-111.84522898530405, 33.561592546308965], [-112.00680781469596, 33.561592546308965]]
[[-112.15488774452213, 33.38106305369104], [-111.99339425547787, 33.38106305369104], [-111.99339425547787, 33.51581034630897], [-112.15488774452213, 33.51581034630897]]
[[-111.84383224726236, 34.80058625369104], [-111.67960075273766, 34.80058625369104], [-111.67960075273766, 34.93533354630897], [-111.84383224726236, 34.93533354630897]]


In [ ]:
[[33.426845253691035, -112.00680781469596], [33.426845253691035, -111.84522898530405], [33.561592546308965, -111.84522898530405], [33.561592546308965, -112.00680781469596]]

[[-112.0068, 33.4268],
 [-111.8452, 33.4268],
 [-111.8452, 33.5615],
 [-112.0068, 33.5615]]

In [ ]:
 [[-112.0068, 33.4268], [-111.8452, 33.4268], [-111.8452, 33.5615], [-112.0068, 33.5615]]